---
## Kendi API mizin son adımına geldik.Bundan sonraki bütün iyileştirme adımları bu dosya içerisinde olacak.
---

# ⚙️ AŞAMA 9 — Dynamic API Core Geliştirme Adımları

## 1️⃣ Route Sistemini Dinamik Hale Getirme
- Statik `ROUTES` sözlüğü yerine dinamik bir route kayıt mekanizması oluşturulacak.  
- `@route(method, path)` dekoratörü tanımlanarak yeni endpoint eklemek basitleştirilecek.  
- Böylece her yeni fonksiyon, otomatik olarak router’a kaydolacak.



## 2️⃣ Otomatik JSON Dönüş Sistemi
- Handler fonksiyonları artık ham string yerine Python nesnesi (`dict`, `list`, vs.) dönecek.  
- Sunucu, dönen veriyi otomatik olarak JSON’a çevirip uygun `Content-Type` başlığıyla cevap verecek.  
- İsteğe göre HTML veya TEXT tipinde yanıt döndürmek için yardımcı dönüş tipleri eklenecek.



## 3️⃣ Merkezi Hata Yönetimi
- Her endpoint çağrısı `try/except` bloğuna alınarak sistematik hata yakalama uygulanacak.  
- Hatalar standartlaştırılmış JSON formatında döndürülecek (`error`, `details`, `status`).  
- 400 (Bad Request), 404 (Not Found) ve 500 (Internal Server Error) için ortak şablonlar oluşturulacak.


## 4️⃣ Loglama ve İstek Sayaçları
- Her isteğin zamanı, IP adresi, path bilgisi, status kodu ve thread adı loglanacak.  
- Terminalde okunabilir renkli log formatı kullanılacak.  
- Toplam istek sayısı tutulacak, sistemin performansı izlenebilir olacak.



## 5️⃣ Health ve Info Endpoint’leri
- `/health`: Sunucunun aktif olduğunu, thread sayısını ve istek sayısını gösterecek.  
- `/info`: Sunucuda kayıtlı tüm rotaları, uptime bilgisini ve genel durumu gösterecek.  
- Bu endpoint’ler otomatik olarak framework’ün parçası olacak.



## 6️⃣ Otomatik JSON Parsing
- POST isteklerinde gelen `application/json` içerik otomatik olarak `dict`’e dönüştürülecek.  
- Handler artık manuel `json.loads()` yapmadan `body` parametresini Python nesnesi olarak alacak.  
- Yanlış formatlı JSON’lar sistem tarafından otomatik reddedilecek.


## 7️⃣ Thread ve Performans Yönetimi
- Her bağlantı ayrı thread’de işlendiği için thread yönetimi geliştirilecek.  
- Aktif thread sayısı izlenecek, gerektiğinde sınırlama (örneğin max 50 thread) getirilebilecek.  
- Thread havuzu (thread pool) altyapısına geçiş için temel atılacak.



## 8️⃣ Modüler Yapı ve Genişletilebilirlik
- API yapısı dosyalara bölünebilir hale getirilecek (`routes.py`, `server.py`, `utils.py` vb.).  
- Yeni endpoint’ler sadece bir dosyada tanımlanarak sisteme dahil edilebilecek.  
- İleri aşamalarda middleware, auth ve statik dosya servisi gibi özellikler kolayca eklenebilecek.



Bu adımlar tamamlandığında, elindeki “manual socket API” artık kendi mini-framework’üne dönüşmüş olacak — hem öğretici, hem de FastAPI benzeri bir yapının iç mantığını birebir modellemiş hale geleceksin.


------

# 🧩 Adım 1 – Dinamik Route Sistemi (Decorator Mantığı)

* Şu anda API’mizde endpoint’ler sabit bir sözlükte (ROUTES = {("GET","/"): handle_get_root, ...}) tanımlı.

* Bu, küçük sistemlerde işe yarar ama her yeni route eklediğinde bu tabloya manuel ekleme yapmak zorundasın.
* Modern web framework’leri (ör. Flask, FastAPI) bu işi dekoratörler aracılığıyla yapar.

* Bir “decorator”, fonksiyonların üstüne yazılan bir Python yapısıdır — ek işlev kazandırır.
* Biz de @route("GET", "/") gibi bir dekoratör tanımlayıp, o fonksiyonun otomatik olarak ROUTES tablosuna eklenmesini sağlayacağız.

Bu sayede yeni endpoint eklemek şöyle kolaylaşır:

```python
@route("GET", "/hello")
def say_hello():
    return {"message": "Merhaba!"}
```

* Ve bu fonksiyon kendini router’a kaydeder; ROUTES’a manuel ekleme gerekmez.

Yani bu adımın özü:
* El ile route eklemeyi bırakıyoruz, sistem kendi kendini tanıyor.

## 🧩  Dinamik Route Sistemi (Temel Kod)

In [1]:
# ==================================================
# 🧩 Dinamik Route Sistemi (Decorator Tabanlı)
# ==================================================

ROUTES = {}  # Tüm endpoint’leri burada toplayacağız


# --- Dekoratör Tanımı ---
def route(method, path):
    """
    Bir endpoint tanımlamak için kullanılır.
    Örnek: @route("GET", "/hello")
    """
    def decorator(func):
        ROUTES[(method.upper(), path)] = func
        print(f"📍 Route eklendi → {method.upper()} {path} -> {func.__name__}")
        return func
    return decorator


# --- Örnek Kullanımlar ---
@route("GET", "/")
def home():
    return {"message": "Hoş geldin! Bu dinamik bir route."}


@route("POST", "/sum")
def sum_numbers():
    return {"info": "Bu POST route örneği!"}


# --- Router Kontrolü ---
if __name__ == "__main__":
    print("\n📜 Kayıtlı route'lar:")
    for key, func in ROUTES.items():
        print(f"{key} → {func.__name__}")


📍 Route eklendi → GET / -> home
📍 Route eklendi → POST /sum -> sum_numbers

📜 Kayıtlı route'lar:
('GET', '/') → home
('POST', '/sum') → sum_numbers


### 🧠 Ne Yaptık?

* route(method, path) adında bir decorator tanımladık.

* Bu decorator, üstüne yazıldığı her fonksiyonu ROUTES adlı sözlüğe otomatik kaydediyor.

### Artık bir sonraki adım aşama 5' e bu yapıyı entegre etmek.Bundan sonraki kodda şunları yapacağız ; 
* route() dekoratörünü ekleyeceğiz.

* Statik ROUTES sözlüğünü kaldıracağız.

* Endpoint’leri @route ile kaydedeceğiz.

* handle_client() fonksiyonunda route çağrısı aynı şekilde kalacak.

## 🧠 AŞAMA 5 —  Dinamik Route Sistemi Entegre Edilmiş API

In [ ]:
# ==========================================================
# 🧩 AŞAMA 9.3 — Statik Dosya Servisi Entegre
# ==========================================================

import socket
import threading
import json
import os

HOST = "127.0.0.1"
PORT = 8092
BUF = 4096

# ==========================================================
# 1️⃣ Dinamik Router Sistemi
# ==========================================================
ROUTES = {}


def route(method, path):
    """Fonksiyonları otomatik olarak ROUTES tablosuna ekleyen dekoratör."""
    def decorator(func):
        ROUTES[(method.upper(), path)] = func
        print(f"📍 Route eklendi → {method.upper()} {path} -> {func.__name__}")
        return func
    return decorator


# ==========================================================
# 2️⃣ Yardımcı Fonksiyonlar
# ==========================================================
def make_response(status, ctype, body):
    status_text = {
        200: "OK",
        400: "Bad Request",
        404: "Not Found",
        500: "Internal Server Error"
    }.get(status, "OK")
    response = f"""HTTP/1.1 {status} {status_text}
Content-Type: {ctype}; charset=utf-8
Content-Length: {len(body)}

{body}"""
    return response.encode("utf-8")


def parse_request(conn):
    """İstekleri (header + body) okur ve parçalar."""
    data = b""
    while True:
        chunk = conn.recv(BUF)
        if not chunk:
            break
        data += chunk
        if b"\r\n\r\n" in data and len(data.split(b"\r\n\r\n", 1)[1]) > 0:
            break

    raw = data.decode("utf-8", errors="ignore")
    headers_part, _, body = raw.partition("\r\n\r\n")
    lines = headers_part.split("\r\n")

    method, path, _ = lines[0].split()
    path = path.rstrip("/")  # sondaki /'yi kaldır

    headers = {}
    for line in lines[1:]:
        if ":" in line:
            key, value = line.split(":", 1)
            headers[key.strip().lower()] = value.strip()

    return method, path, headers, body


# ==========================================================
# 3️⃣ Endpoint’ler (Dinamik)
# ==========================================================
@route("GET", "/")
def handle_get_root():
    body = "<h1>Dynamic Threaded API ⚙️</h1><p>Route’lar artık otomatik tanınıyor!</p>"
    return 200, "text/html", body


@route("POST", "/sum")
def handle_post_sum(headers, body):
    if headers.get("content-type", "") != "application/json":
        err = json.dumps({"error": "Content-Type application/json olmalı"}, ensure_ascii=False)
        return 400, "application/json", err

    try:
        data = json.loads(body)
    except json.JSONDecodeError:
        err = json.dumps({"error": "Geçersiz JSON"}, ensure_ascii=False)
        return 400, "application/json", err

    if not all(k in data for k in ("a", "b")):
        err = json.dumps({"error": "JSON içinde 'a' ve 'b' anahtarları olmalı"}, ensure_ascii=False)
        return 400, "application/json", err

    try:
        a, b = float(data["a"]), float(data["b"])
        toplam = a + b
    except Exception:
        err = json.dumps({"error": "a ve b sayısal olmalı"}, ensure_ascii=False)
        return 400, "application/json", err

    result = json.dumps({
        "a": a,
        "b": b,
        "sum": toplam,
        "thread": threading.current_thread().name
    }, ensure_ascii=False)
    return 200, "application/json", result


# ==========================================================
# 4️⃣ Statik Dosya Servisi
# ==========================================================
STATIC_DIR = r"C:\Users\hdgn5\OneDrive\Masaüstü\Kendi API'mız\- API Geliştirne Adımları -\Part 3"

def serve_static(path):
    """İstenen dosya STATIC_DIR içinde varsa gönderir."""
    if path in ("", "/"):
        path = "text.html"  # Varsayılan dosya

    full_path = os.path.join(STATIC_DIR, path.lstrip("/"))
    if not os.path.exists(full_path):
        return 404, "text/html", f"<h1>404 Not Found</h1><p>{path} bulunamadı.</p>"

    # MIME türünü belirle
    if full_path.endswith(".html"):
        ctype = "text/html"
    elif full_path.endswith(".css"):
        ctype = "text/css"
    elif full_path.endswith(".js"):
        ctype = "application/javascript"
    else:
        ctype = "application/octet-stream"

    with open(full_path, "r", encoding="utf-8", errors="ignore") as f:
        body = f.read()

    print(f"📄 Aranan dosya: {path}")
    print(f"📁 STATIC_DIR: {STATIC_DIR}")
    print(f"🔍 Tam yol: {os.path.join(STATIC_DIR, path.lstrip('/'))}")


    return 200, ctype, body


# ==========================================================
# 5️⃣ İstekleri İşleyen Thread Fonksiyonu
# ==========================================================
def handle_client(conn, addr):
    print(f"📡 Yeni bağlantı: {addr} | Thread: {threading.current_thread().name}")
    try:
        method, path, headers, body = parse_request(conn)
        key = (method.upper(), path)
        print(f"🔎 İstek: {key}")

        if key in ROUTES:
            handler = ROUTES[key]
            if method == "GET":
                status, ctype, resp_body = handler()
            else:
                status, ctype, resp_body = handler(headers, body)
        else:
            # Dinamik route yoksa statik dosya olarak dene
            status, ctype, resp_body = serve_static(path)

    except Exception as e:
        print(f"⚠️ Hata: {e}")
        status, ctype, resp_body = 500, "application/json", json.dumps({"error": str(e)}, ensure_ascii=False)

    response = make_response(status, ctype, resp_body)
    conn.sendall(response)
    conn.close()
    print(f"✅ {addr} isteği işlendi → {threading.current_thread().name}")



def start_server():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as srv:
        srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        srv.bind((HOST, PORT))
        srv.listen(5)
        print(f"\n🚀 Statik + Dinamik Multi-threaded API çalışıyor: http://{HOST}:{PORT}\n")
        while True:
            conn, addr = srv.accept()
            t = threading.Thread(target=handle_client, args=(conn, addr))
            t.start()

# 🔥 Arka planda çalıştır:
threading.Thread(target=start_server, daemon=True).start()


📍 Route eklendi → GET / -> handle_get_root
📍 Route eklendi → POST /sum -> handle_post_sum

🚀 Statik + Dinamik Multi-threaded API çalışıyor: http://127.0.0.1:8092



📡 Yeni bağlantı: ('127.0.0.1', 62807) | Thread: Thread-33 (handle_client)
📡 Yeni bağlantı: ('127.0.0.1', 62013) | Thread: Thread-34 (handle_client)
🔎 İstek: ('GET', '')
📄 Aranan dosya: text.html
📁 STATIC_DIR: C:\Users\hdgn5\OneDrive\Masaüstü\Kendi API'mız\- API Geliştirne Adımları -\Part 3
🔍 Tam yol: C:\Users\hdgn5\OneDrive\Masaüstü\Kendi API'mız\- API Geliştirne Adımları -\Part 3\text.html
✅ ('127.0.0.1', 62807) isteği işlendi → Thread-33 (handle_client)
📡 Yeni bağlantı: ('127.0.0.1', 51204) | Thread: Thread-35 (handle_client)
🔎 İstek: ('GET', '/text.html')
📄 Aranan dosya: /text.html
📁 STATIC_DIR: C:\Users\hdgn5\OneDrive\Masaüstü\Kendi API'mız\- API Geliştirne Adımları -\Part 3
🔍 Tam yol: C:\Users\hdgn5\OneDrive\Masaüstü\Kendi API'mız\- API Geliştirne Adımları -\Part 3\text.html
✅ ('127.0.0.1', 62013) isteği işlendi → Thread-34 (handle_client)
⚠️ Hata: not enough values to unpack (expected 3, got 0)
✅ ('127.0.0.1', 51204) isteği işlendi → Thread-35 (handle_client)


---

## 🧩 AŞAMA 6 — Otomatik JSON Dönüş Sistemi (Auto-Response Layer)
### 🧠 Amaç

* Şu an her endpoint şu biçimde dönüyor:
```BASH
return 200, "application/json", json.dumps({...})
```


* Bu çok tekrarlı. Her json.dumps, her Content-Type, her status kodu manuel.
* Yeni sistemde handler yalnızca Python nesnesi dönecek:

>return {"a": 5, "b": 7, "sum": 12}


Sunucu otomatik olarak:

* türünü algılayacak (dict → JSON, str → HTML/text),

* uygun Content-Type seçecek,

* 200 OK statüsünü varsayılan yapacak.

### ⚙️ Ne Kazandıracak

* Kod Temizliği → endpoint başına 2-3 satır azalır.

* Otomatik Tür Algısı → ister JSON ister HTML döndür, framework halleder.

* Hata Yönetimi Kolaylaşır → aynı make_response tüm durumları standartlaştırır.

* Genişleme Altyapısı → ileride HTML(), TEXT(), JSON() gibi özel yanıt tipleri eklenebilir.

### 📐 Nasıl Uygulayacağız

* make_response() fonksiyonunu akıllandıracağız.

* Dönen değer tuple ise eskisi gibi davran.

* Dönen değer dict ise otomatik JSON’a çevir.

* Dönen değer str ise text/html olarak gönder.

* Endpoint’ler sadece return {"msg": "ok"} yazacak.

* Sunucu, hata oluştuğunda yine 500 Internal Server Error dönecek.

### 🚀 Sonuç,
* Kendi mini-FastAPI çekirdeğimizin “response layer”ını tamamlamış olacağız.

## 🎯 Küçük Deneme:

* Bu örnek sadece “otomatik dönüş tipi” sistemini anlatıyor.
* Henüz thread, socket, route yok — sadece fonksiyon davranışını görelim.

In [8]:
import json
from typing import Any

def make_response_auto(data: Any):
    """Dönen verinin türüne göre uygun Content-Type ve Body üretir."""
    if isinstance(data, tuple):
        # Eski sistemle uyumlu: (status, content_type, body)
        status, ctype, body = data
    elif isinstance(data, dict):
        # Otomatik JSON dönüşü
        status = 200
        ctype = "application/json"
        body = json.dumps(data, ensure_ascii=False)
    elif isinstance(data, str):
        # Düz HTML veya metin
        status = 200
        ctype = "text/html"
        body = data
    else:
        # Bilinmeyen tür
        status = 500
        ctype = "application/json"
        body = json.dumps({"error": f"Desteklenmeyen tür: {type(data)}"}, ensure_ascii=False)
    return f"HTTP/1.1 {status} OK\nContent-Type: {ctype}\n\n{body}"


In [9]:
print(make_response_auto({"msg": "Merhaba"}))
print()
print(make_response_auto("<h1>Selam HTML</h1>"))
print()
print(make_response_auto((404, "text/html", "<h1>Sayfa Yok</h1>")))


HTTP/1.1 200 OK
Content-Type: application/json

{"msg": "Merhaba"}

HTTP/1.1 200 OK
Content-Type: text/html

<h1>Selam HTML</h1>

HTTP/1.1 404 OK
Content-Type: text/html

<h1>Sayfa Yok</h1>


## Şimdi entegresini yapalım ;

### 🧩 AŞAMA 6 — Akıllı make_response() Entegreli API

In [ ]:
# ==========================================================
# 🧩 AŞAMA 9.4.1 — Auto Response + Binary Static + Doğru Content-Length
# ==========================================================

import socket
import threading
import json
import os
import mimetypes
from typing import Any

# ---- Ayarlar ----
HOST = "127.0.0.1"
PORT = 8094
BUF = 4096

# Statik dosyaların bulunduğu klasör (senin verdiğin yol)
STATIC_DIR = r"C:\Users\hdgn5\OneDrive\Masaüstü\Kendi API'mız\- API Geliştirne Adımları -\Part 3"

mimetypes.init()

# ==========================================================
# 1) Dinamik Router (Decorator)
# ==========================================================
ROUTES = {}

def route(method: str, path: str):
    def decorator(func):
        ROUTES[(method.upper(), path)] = func
        print(f"📍 Route eklendi → {method.upper()} {path} -> {func.__name__}")
        return func
    return decorator

# ==========================================================
# 2) Auto Response (dict→JSON, str→HTML, bytes→binary)
# ==========================================================
def make_response_auto(data: Any):
    """Her türlü dönen veriyi (status,ctype,body) üçlüsüne normalize eder."""
    if isinstance(data, tuple):
        # Eski stil: (status, ctype, body)
        status, ctype, body = data
        return status, ctype, body

    if isinstance(data, dict):
        # Otomatik JSON
        return 200, "application/json", json.dumps(data, ensure_ascii=False)

    if isinstance(data, str):
        # Düz HTML/metin
        return 200, "text/html", data

    if isinstance(data, (bytes, bytearray)):
        # Binary içerik
        return 200, "application/octet-stream", bytes(data)

    # Bilinmeyen tür
    return 500, "application/json", json.dumps({"error": f"unsupported type: {type(data)}"}, ensure_ascii=False)

def build_http_response(status: int, ctype: str, body):
    """HTTP cevabını **bytes** olarak oluşturur (Content-Length byte cinsinden)."""
    status_text = {
        200: "OK",
        400: "Bad Request",
        404: "Not Found",
        500: "Internal Server Error"
    }.get(status, "OK")

    # body → bytes
    if isinstance(body, str):
        body_bytes = body.encode("utf-8")
    elif isinstance(body, (bytes, bytearray)):
        body_bytes = bytes(body)
    else:
        # Emniyet: dict/list vs. gelirse JSON'a çevir
        body_bytes = json.dumps(body, ensure_ascii=False).encode("utf-8")
        ctype = "application/json"

    headers = (
        f"HTTP/1.1 {status} {status_text}\r\n"
        f"Content-Type: {ctype}; charset=utf-8\r\n"
        f"Content-Length: {len(body_bytes)}\r\n"
        f"Connection: close\r\n"
        f"\r\n"
    ).encode("ascii")

    return headers + body_bytes

# ==========================================================
# 3) HTTP İstek Parsing (Content-Length destekli)
# ==========================================================
def _recv_until_headers_end(conn):
    data = b""
    while b"\r\n\r\n" not in data:
        chunk = conn.recv(BUF)
        if not chunk:
            break
        data += chunk
    return data

def parse_request(conn):
    """Header'ları okur, Content-Length varsa body'yi tam boyda çeker."""
    header_block = _recv_until_headers_end(conn)
    if not header_block:
        # Boş istek
        return "GET", "/", {}, ""

    headers_part, _, tail = header_block.partition(b"\r\n\r\n")
    head_lines = headers_part.decode("utf-8", errors="ignore").split("\r\n")

    # Request line
    method, path, _ = head_lines[0].split()
    # Headerlar
    headers = {}
    for line in head_lines[1:]:
        if ":" in line:
            k, v = line.split(":", 1)
            headers[k.strip().lower()] = v.strip()

    # Content-Length varsa geri kalan body'yi tamamla
    content_length = int(headers.get("content-length", "0")) if headers else 0
    body_bytes = tail
    to_read = max(0, content_length - len(body_bytes))
    while to_read > 0:
        chunk = conn.recv(min(BUF, to_read))
        if not chunk:
            break
        body_bytes += chunk
        to_read -= len(chunk)

    # Path normalize
    path = (path or "/").rstrip("/")  # "/sum/" → "/sum", "" → "/"
    if path == "":
        path = "/"

    try:
        body = body_bytes.decode("utf-8", errors="ignore")
    except:
        body = ""

    return method, path, headers, body

# ==========================================================
# 4) Endpoint’ler
# ==========================================================
@route("GET", "/")
def handle_root():
    return "<h1>Auto Response API ⚙️</h1><p>Handler'lar Python objesi döndürüyor; Content-Length byte cinsinden.</p>"

@route("POST", "/sum")
def handle_sum(headers, body):
    try:
        if headers.get("content-type", "").lower().startswith("application/json"):
            data = json.loads(body)
        else:
            return (400, "application/json", json.dumps({"error": "Content-Type application/json olmalı"}, ensure_ascii=False))
        a, b = float(data["a"]), float(data["b"])
        return {"a": a, "b": b, "sum": a + b, "thread": threading.current_thread().name}
    except Exception as e:
        return (400, "application/json", json.dumps({"error": str(e)}, ensure_ascii=False))

@route("GET", "/sum")
def handle_sum_get():
    # Basit bir HTML formu: tarayıcıdan test için kullanışlı
    return """<!DOCTYPE html>
<html lang="tr"><meta charset="utf-8"><title>Sum Test</title>
<body style="font-family:ui-sans-serif">
  <h3>Toplama (POST /sum)</h3>
  <input id="a" type="number" placeholder="a"> +
  <input id="b" type="number" placeholder="b">
  <button onclick="send()">Topla</button>
  <pre id="out"></pre>
  <script>
    async function send(){
      const a = parseFloat(document.getElementById('a').value||0);
      const b = parseFloat(document.getElementById('b').value||0);
      const res = await fetch('/sum', {
        method: 'POST',
        headers: {'Content-Type':'application/json'},
        body: JSON.stringify({a,b})
      });
      document.getElementById('out').textContent = await res.text();
    }
  </script>
</body></html>"""


# ==========================================================
# 5) Statik Dosya Servisi (binary okuma + doğru MIME)
# ==========================================================
def serve_static(path: str):
    # / → varsayılan dosya
    if path in ("", "/"):
        path = "text.html"

    full_path = os.path.join(STATIC_DIR, path.lstrip("/"))

    if not os.path.exists(full_path) or not os.path.isfile(full_path):
        return 404, "text/html", "<h1>404 Not Found</h1>"

    ctype, _ = mimetypes.guess_type(full_path)
    if not ctype:
        ctype = "application/octet-stream"

    with open(full_path, "rb") as f:
        body = f.read()

    return 200, ctype, body

# ==========================================================
# 6) İstek İşleyici (Thread başına)
# ==========================================================
def handle_client(conn, addr):
    print(f"📡 Yeni bağlantı: {addr} | Thread: {threading.current_thread().name}")
    try:
        method, path, headers, body = parse_request(conn)
        key = (method.upper(), path)
        print(f"🔎 İstek: {key}")

        if key in ROUTES:
            handler = ROUTES[key]
            raw = handler() if method == "GET" else handler(headers, body)
        else:
            raw = serve_static(path)

        status, ctype, resp_body = make_response_auto(raw)
    except Exception as e:
        print(f"⚠️ Hata: {e}")
        status, ctype, resp_body = 500, "application/json", json.dumps({"error": str(e)}, ensure_ascii=False)

    conn.sendall(build_http_response(status, ctype, resp_body))
    conn.close()
    print(f"✅ {addr} isteği işlendi → {threading.current_thread().name}")

# ==========================================================
# 7) Sunucu Başlatma
# ==========================================================
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as srv:
    srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    srv.bind((HOST, PORT))
    srv.listen(64)
    print(f"\n🚀 Auto Response + Static Server çalışıyor: http://{HOST}:{PORT}\n")

    while True:
        conn, addr = srv.accept()
        t = threading.Thread(target=handle_client, args=(conn, addr))
        t.start()


📍 Route eklendi → GET / -> handle_root
📍 Route eklendi → POST /sum -> handle_sum
📍 Route eklendi → GET /sum -> handle_sum_get

🚀 Auto Response + Static Server çalışıyor: http://127.0.0.1:8094

📡 Yeni bağlantı: ('127.0.0.1', 52202) | Thread: Thread-3 (handle_client)
🔎 İstek: ('GET', '/')
✅ ('127.0.0.1', 52202) isteği işlendi → Thread-3 (handle_client)
📡 Yeni bağlantı: ('127.0.0.1', 49832) | Thread: Thread-4 (handle_client)
🔎 İstek: ('GET', '/sum')
✅ ('127.0.0.1', 49832) isteği işlendi → Thread-4 (handle_client)
📡 Yeni bağlantı: ('127.0.0.1', 54211) | Thread: Thread-5 (handle_client)
📡 Yeni bağlantı: ('127.0.0.1', 63441) | Thread: Thread-6 (handle_client)
🔎 İstek: ('POST', '/sum')
✅ ('127.0.0.1', 54211) isteği işlendi → Thread-5 (handle_client)
🔎 İstek: ('POST', '/sum')
✅ ('127.0.0.1', 63441) isteği işlendi → Thread-6 (handle_client)
📡 Yeni bağlantı: ('127.0.0.1', 58085) | Thread: Thread-7 (handle_client)
🔎 İstek: ('POST', '/sum')
✅ ('127.0.0.1', 58085) isteği işlendi → Thread-7 (handle_

-----

## Aşama 7 – Global Error Handler

## 🧠 Mantık Anlatımı

* Şu an bir endpoint’te hata olursa şunu yapıyoruz:
```python
except Exception as e:
    return (400, "application/json", json.dumps({"error": str(e)}))
```


* Bu, çok tekrarlanıyor.
* Her fonksiyona try/except koymak hem sıkıcı hem hataya açık.

### Global Error Handler sistemiyle:

* Tüm hatalar tek yerde yakalanır.

* Dönüş tipi otomatik belirlenir (JSON veya HTML).

* Kod temiz kalır.

## 🔧 Nasıl Çalışır?

### Yeni sistemin temel fikri:

>handle_client() içinde tüm try/except’leri tek bir “error middleware” içine alırız.

### Herhangi bir endpoint patlarsa:

* hata türüne göre status code belirlenir (ör. ValueError → 400, FileNotFoundError → 404, Exception → 500),

* body otomatik oluşturulur:

>{"error": "mesaj", "type": "ValueError"}


* Artık endpoint’lerin içinde try/except yok.
* Onlar sadece ana işi yapar.
* Hata olursa, sistem kendisi 500 döner.

## 🎯 Küçük Örnek (Bağımsız Test Kod)

Aşağıdaki mini örnek sistemi anlatır:

In [1]:
def handle_request(path):
    if path == "/":
        return "<h1> ANA SAYFA </h1>"
    elif path == "/sum":
        raise ValueError("Eksik Parametre")
    else:
        raise FileNotFoundError("Bu sayfa yok")
    
def global_handler(func, *args):
    try:
        result = func(*args)
        return (200, "text/html", result)
    except FileNotFoundError as e:
        return (404, "application/json", {"error": str(e), "type": "FileNotFoundError"})
    except ValueError as e:
        return (400, "application/json", {"error": str(e), "type": "ValueError"})
    except Exception as e:
        return (500, "application/json", {"error": str(e), "type": "Exception"})

print(global_handler(handle_request, "/sum"))
print(global_handler(handle_request, "/abc"))

(400, 'application/json', {'error': 'Eksik Parametre', 'type': 'ValueError'})
(404, 'application/json', {'error': 'Bu sayfa yok', 'type': 'FileNotFoundError'})


### 💬 Yani Özetle:

* Artık her fonksiyon kendi içinde try/except tutmaz.

* Tek merkezden “hata → uygun cevap” çevirisi yapılır.

* Kod temizlenir, hata çıktıları standartlaşır.

* FastAPI veya Flask’ın ExceptionHandler yapısının sade versiyonunu kurmuş olacağız.

## 🧩 AŞAMA 7 — Global Error Handler Entegre Edilmiş API

In [ ]:
import socket , threading 
import json , os
import mimetypes
from typing import Any

HOST , PORT , BUF = "127.0.0.1" , 8092 , 4096

STATIC_DIR  = r"C:\Users\hdgn5\OneDrive\Masaüstü\Kendi API'mız\- API Geliştirne Adımları -\Part 3"

mimetypes.init()

ROUTES = {}

def route(method , path):
    def decorator(func):
        ROUTES[(method.upper() , path)] = func
        print(f"Route eklendi -> {method.upper()} {path} -> {func.__name__}")
        return func
    return decorator


def build_http_response(status: int, ctype: str, body):
    if isinstance(body, str):
        body_bytes = body.encode("utf-8")
    elif isinstance(body, (bytes, bytearray)):
        body_bytes = bytes(body)
    else:
        body_bytes = json.dumps(body, ensure_ascii=False).encode("utf-8")
        ctype = "application/json"

    status_text = {
        200: "OK", 400: "Bad Request", 404: "Not Found", 500: "Internal Server Error"
    }.get(status, "OK")

    headers = (
        f"HTTP/1.1 {status} {status_text}\r\n"
        f"Content-Type: {ctype}; charset=utf-8\r\n"
        f"Content-Length: {len(body_bytes)}\r\n"
        f"Connection: close\r\n"
        f"\r\n"
    ).encode("ascii")

    return headers + body_bytes

def global_error_handler(func, *args):
    try:
        return func(*args)
    except FileNotFoundError as e:
        return (404, "application/json", json.dumps({"error": str(e), "type": "FileNotFoundError"}, ensure_ascii=False))
    except ValueError as e:
        return (400, "application/json", json.dumps({"error": str(e), "type": "ValueError"}, ensure_ascii=False))
    except KeyError as e:
        return (400, "application/json", json.dumps({"error": f"Eksik anahtar: {e}", "type": "KeyError"}, ensure_ascii=False))
    except Exception as e:
        return (500, "application/json", json.dumps({"error": str(e), "type": type(e).__name__}, ensure_ascii=False))
    
def parse_request(conn):
    data = b""
    while b"\r\n\r\n" not in data:
        chunk = conn.recv(BUF)
        if not chunk:
            break
        data += chunk

    headers_part, _, body_bytes = data.partition(b"\r\n\r\n")
    lines = headers_part.decode("utf-8", errors="ignore").split("\r\n")
    method, path, _ = lines[0].split()
    headers = {}
    for line in lines[1:]:
        if ":" in line:
            key, value = line.split(":", 1)
            headers[key.strip().lower()] = value.strip()

    content_length = int(headers.get("content-length", "0")) if headers else 0
    while len(body_bytes) < content_length:
        body_bytes += conn.recv(BUF)

    body = body_bytes.decode("utf-8", errors="ignore")
    return method, path.rstrip("/") or "/", headers, body

# ==========================================================
# 5️⃣ Endpoint’ler
# ==========================================================
@route("GET", "/")
def root():
    return "<h1>Global Error Handler ⚙️</h1><p>Artık her hata düzgün JSON formatında dönüyor!</p>"

@route("POST", "/sum")
def sum_numbers(headers, body):
    data = json.loads(body)
    a, b = float(data["a"]), float(data["b"])  # eksikse KeyError yakalanır
    return {"sum": a + b}

@route("POST", "/sum/plain")
def sum_plain(headers, body):
    data = json.loads(body)
    a, b = float(data["a"]), float(data["b"])
    return (200, "text/plain", str(a + b))

# ==========================================================
# 6️⃣ Statik Dosya Servisi
# ==========================================================
def serve_static(path):
    if path in ("", "/"):
        path = "text.html"
    full_path = os.path.join(STATIC_DIR, path.lstrip("/"))
    if not os.path.exists(full_path):
        raise FileNotFoundError(f"{path} bulunamadı.")
    ctype, _ = mimetypes.guess_type(full_path)
    if not ctype:
        ctype = "application/octet-stream"
    with open(full_path, "rb") as f:
        body = f.read()
    return 200, ctype, body

def handle_client(conn, addr):
    print(f"📡 Yeni bağlantı: {addr} | Thread: {threading.current_thread().name}")
    try:
        method, path, headers, body = parse_request(conn)
        key = (method.upper(), path)
        print(f"🔎 İstek: {key}")

        if key in ROUTES:
            handler = ROUTES[key]
            raw = global_error_handler(handler, headers, body) if method == "POST" else global_error_handler(handler)
        else:
            raw = global_error_handler(serve_static, path)

        status, ctype, resp_body = make_response_auto(raw)
    except Exception as e:
        status, ctype, resp_body = 500, "application/json", json.dumps({"error": str(e)}, ensure_ascii=False)

    conn.sendall(build_http_response(status, ctype, resp_body))
    conn.close()
    print(f"✅ {addr} isteği işlendi → {threading.current_thread().name}")

# ==========================================================
# 8️⃣ Sunucu Başlatma
# ==========================================================
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as srv:
    srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    srv.bind((HOST, PORT))
    srv.listen(64)
    print(f"\n🚀 Global Error Handler API çalışıyor: http://{HOST}:{PORT}\n")

    while True:
        conn, addr = srv.accept()
        t = threading.Thread(target=handle_client, args=(conn, addr))
        t.start()


## ✅ Şimdi ne değişti?

| Durum              | Eski Davranış  | Yeni Davranış                                                          |
| ------------------ | -------------- | ---------------------------------------------------------------------- |
| Eksik “a” veya “b” | Sunucu çökerdi | `400 Bad Request` + `{"error":"Eksik anahtar: 'a'","type":"KeyError"}` |
| JSON hatalı        | Kod kırılırdı  | `400` + `"ValueError"`                                                 |
| Dosya bulunmadı    | Boş sayfa      | `404` + `"FileNotFoundError"`                                          |
| Bilinmeyen hata    | Crash          | `500` + `"Exception"`                                                  |


----

# Aşama 8 – Renkli Log Sistemi + Zaman Damgası.

### 🧠 Mantık Anlatımı

Şu an loglarımız şöyle görünüyor:

```html
📡 Yeni bağlantı: ('127.0.0.1', 54122)
🔎 İstek: ('POST', '/sum')
✅ ... işlendi
```


#### Ama büyük sistemlerde renk kodları ve zaman damgası çok önemli:

Mavi → bilgi

Yeşil → başarı

Sarı → uyarı

Kırmızı → hata

#### Böylece 1000’lerce log arasında bir bakışta anlarız:

Ne zaman istek geldi

Hangi endpoint çağrıldı

Ne hata oldu

İşlem süresi ne kadar sürdü

## 🎨 Renkli Loglama Nasıl Yapılır?

* Python terminali ANSI renk kodlarını destekler.
Renklerin temeli:
```python
RESET = "\033[0m"
RED = "\033[91m"
GREEN = "\033[92m"
YELLOW = "\033[93m"
BLUE = "\033[94m"
CYAN = "\033[96m"
```


#### Zaman etiketi için:
```pyhon
from datetime import datetime
now = datetime.now().strftime("%H:%M:%S")
```


Kullanım örneği:

```python
print(f"{BLUE}[{now}]{RESET} {GREEN}✅ İstek işlendi!{RESET}")

### 🔍 Küçük Test Örneği

Aşağıdaki örnek sadece sistemi tanıtıyor:

In [4]:
from datetime import datetime

RESET = "\033[0m"
RED = "\033[91m"
GREEN = "\033[92m"
YELLOW = "\033[93m"
BLUE = "\033[94m"
CYAN = "\033[96m"

def log(msg, color=BLUE, symbol="💬"):
    now = datetime.now().strftime("%H:%M:%S")
    print(f"{color}[{now}] {symbol} {msg}{RESET}")

log("Sunucu başlatıldı", CYAN, "🚀")
log("Yeni bağlantı alındı", BLUE, "📡")
log("İstek işlendi", GREEN, "✅")
log("Hatalı istek", RED, "❌")
log("Uyarı: Bağlantı yavaş", YELLOW, "⚠️")


[23:37:55] 🚀 Sunucu başlatıldı
[23:37:55] 📡 Yeni bağlantı alındı
[23:37:55] ✅ İstek işlendi
[23:37:55] ❌ Hatalı istek
[23:37:55] ⚠️ Uyarı: Bağlantı yavaş


## 🧩 AŞAMA 8 — Renkli Log Sistemi + Zaman Damgası Entegreli API

In [ ]:
# ==========================================================
# 🧩 AŞAMA 9.6 — Renkli Log Sistemi + Zaman Damgası
# ==========================================================

import socket
import threading
import json
import os
import mimetypes
from datetime import datetime
from typing import Any

# ---- Genel Ayarlar ----
HOST = "127.0.0.1"
PORT = 8096
BUF = 4096
STATIC_DIR = r"C:\Users\hdgn5\OneDrive\Masaüstü\Kendi API'mız\- API Geliştirne Adımları -\Part 3"

mimetypes.init()

# ==========================================================
# 🎨 1️⃣ Renk Kodları ve Log Fonksiyonu
# ==========================================================
RESET  = "\033[0m"
RED    = "\033[91m"
GREEN  = "\033[92m"
YELLOW = "\033[93m"
BLUE   = "\033[94m"
CYAN   = "\033[96m"

def log(msg: str, color=BLUE, symbol="💬"):
    now = datetime.now().strftime("%H:%M:%S")
    print(f"{color}[{now}] {symbol} {msg}{RESET}")

# ==========================================================
# 2️⃣ Dinamik Route Sistemi
# ==========================================================
ROUTES = {}

def route(method, path):
    def decorator(func):
        ROUTES[(method.upper(), path)] = func
        log(f"Route eklendi → {method.upper()} {path} -> {func.__name__}", CYAN, "📍")
        return func
    return decorator

# ==========================================================
# 3️⃣ Auto Response (dict→JSON, str→HTML, bytes→Binary)
# ==========================================================
def make_response_auto(data: Any):
    if isinstance(data, tuple):
        return data
    if isinstance(data, dict):
        return 200, "application/json", json.dumps(data, ensure_ascii=False)
    if isinstance(data, str):
        return 200, "text/html", data
    if isinstance(data, (bytes, bytearray)):
        return 200, "application/octet-stream", bytes(data)
    return 500, "application/json", json.dumps({"error": f"Unsupported type: {type(data)}"}, ensure_ascii=False)

def build_http_response(status: int, ctype: str, body):
    if isinstance(body, str):
        body_bytes = body.encode("utf-8")
    elif isinstance(body, (bytes, bytearray)):
        body_bytes = bytes(body)
    else:
        body_bytes = json.dumps(body, ensure_ascii=False).encode("utf-8")
        ctype = "application/json"

    status_text = {200:"OK",400:"Bad Request",404:"Not Found",500:"Internal Server Error"}.get(status, "OK")

    headers = (
        f"HTTP/1.1 {status} {status_text}\r\n"
        f"Content-Type: {ctype}; charset=utf-8\r\n"
        f"Content-Length: {len(body_bytes)}\r\n"
        f"Connection: close\r\n"
        f"\r\n"
    ).encode("ascii")

    return headers + body_bytes

# ==========================================================
# 4️⃣ Global Error Handler
# ==========================================================
def global_error_handler(func, *args):
    try:
        return func(*args)
    except FileNotFoundError as e:
        log(f"404 Not Found → {e}", YELLOW, "⚠️")
        return (404, "application/json", json.dumps({"error": str(e), "type": "FileNotFoundError"}, ensure_ascii=False))
    except ValueError as e:
        log(f"400 Bad Request → {e}", YELLOW, "⚠️")
        return (400, "application/json", json.dumps({"error": str(e), "type": "ValueError"}, ensure_ascii=False))
    except KeyError as e:
        log(f"Eksik anahtar → {e}", YELLOW, "⚠️")
        return (400, "application/json", json.dumps({"error": f"Eksik anahtar: {e}", "type": "KeyError"}, ensure_ascii=False))
    except Exception as e:
        log(f"500 Internal Error → {type(e).__name__}: {e}", RED, "❌")
        return (500, "application/json", json.dumps({"error": str(e), "type": type(e).__name__}, ensure_ascii=False))

# ==========================================================
# 5️⃣ HTTP Parsing
# ==========================================================
def parse_request(conn):
    data = b""
    while b"\r\n\r\n" not in data:
        chunk = conn.recv(BUF)
        if not chunk:
            break
        data += chunk

    headers_part, _, body_bytes = data.partition(b"\r\n\r\n")
    lines = headers_part.decode("utf-8", errors="ignore").split("\r\n")
    method, path, _ = lines[0].split()
    headers = {}
    for line in lines[1:]:
        if ":" in line:
            key, value = line.split(":", 1)
            headers[key.strip().lower()] = value.strip()

    content_length = int(headers.get("content-length", "0")) if headers else 0
    while len(body_bytes) < content_length:
        body_bytes += conn.recv(BUF)

    body = body_bytes.decode("utf-8", errors="ignore")
    return method, path.rstrip("/") or "/", headers, body

# ==========================================================
# 6️⃣ Endpoint’ler
# ==========================================================
@route("GET", "/")
def root():
    return "<h1>Renkli Log Sistemi ⚙️</h1><p>Artık her şey renklendi!</p>"

@route("POST", "/sum")
def sum_numbers(headers, body):
    data = json.loads(body)
    a, b = float(data["a"]), float(data["b"])
    return {"sum": a + b}

@route("POST", "/sum/plain")
def sum_plain(headers, body):
    data = json.loads(body)
    a, b = float(data["a"]), float(data["b"])
    return (200, "text/plain", str(a + b))

# ==========================================================
# 7️⃣ Statik Dosya Servisi
# ==========================================================
def serve_static(path):
    if path in ("", "/"):
        path = "text.html"
    full_path = os.path.join(STATIC_DIR, path.lstrip("/"))
    if not os.path.exists(full_path):
        raise FileNotFoundError(f"{path} bulunamadı.")
    ctype, _ = mimetypes.guess_type(full_path)
    if not ctype:
        ctype = "application/octet-stream"
    with open(full_path, "rb") as f:
        body = f.read()
    return 200, ctype, body

# ==========================================================
# 8️⃣ İstek İşleyici (Thread başına)
# ==========================================================
def handle_client(conn, addr):
    log(f"Yeni bağlantı: {addr}", BLUE, "📡")
    try:
        method, path, headers, body = parse_request(conn)
        key = (method.upper(), path)
        log(f"İstek geldi → {key}", YELLOW, "🔎")

        if key in ROUTES:
            handler = ROUTES[key]
            raw = global_error_handler(handler, headers, body) if method == "POST" else global_error_handler(handler)
        else:
            raw = global_error_handler(serve_static, path)

        status, ctype, resp_body = make_response_auto(raw)
        log(f"{status} → {path}", GREEN, "✅")
    except Exception as e:
        log(f"Sunucu hatası: {e}", RED, "💥")
        status, ctype, resp_body = 500, "application/json", json.dumps({"error": str(e)}, ensure_ascii=False)

    conn.sendall(build_http_response(status, ctype, resp_body))
    conn.close()
    log(f"Bağlantı kapatıldı: {addr}", CYAN, "🔚")

# ==========================================================
# 9️⃣ Sunucu Başlatma
# ==========================================================
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as srv:
    srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    srv.bind((HOST, PORT))
    srv.listen(64)
    log(f"Renkli Log API çalışıyor → http://{HOST}:{PORT}", CYAN, "🚀")

    while True:
        conn, addr = srv.accept()
        t = threading.Thread(target=handle_client, args=(conn, addr))
        t.start()


[23:39:37] 📍 Route eklendi → GET / -> root
[23:39:37] 📍 Route eklendi → POST /sum -> sum_numbers
[23:39:37] 📍 Route eklendi → POST /sum/plain -> sum_plain
[23:39:37] 🚀 Renkli Log API çalışıyor → http://127.0.0.1:8096
[23:39:40] 📡 Yeni bağlantı: ('127.0.0.1', 59301)
[23:39:40] 🔎 İstek geldi → ('GET', '/')
[23:39:40] ✅ 200 → /
[23:39:40] 🔚 Bağlantı kapatıldı: ('127.0.0.1', 59301)
[23:39:40] 📡 Yeni bağlantı: ('127.0.0.1', 60846)
[23:39:40] 🔎 İstek geldi → ('GET', '/favicon.ico')
[23:39:40] ⚠️ 404 Not Found → /favicon.ico bulunamadı.
[23:39:40] ✅ 404 → /favicon.ico
[23:39:40] 🔚 Bağlantı kapatıldı: ('127.0.0.1', 60846)
[23:39:43] 📡 Yeni bağlantı: ('127.0.0.1', 64175)
[23:39:43] 🔎 İstek geldi → ('GET', '/sum')
[23:39:43] ⚠️ 404 Not Found → /sum bulunamadı.
[23:39:43] ✅ 404 → /sum
[23:39:43] 📡 Yeni bağlantı: ('127.0.0.1', 61239)
[23:39:43] 🔚 Bağlantı kapatıldı: ('127.0.0.1', 64175)
[23:39:49] 🔎 İstek geldi → ('GET', '/doc')
[23:39:49] ⚠️ 404 Not Found → /doc bulunamadı.
[23:39:49] ✅ 404 → /do

## Aşama 9 – Dosyaya Loglama (Persistent Logger)

### 🧠 Mantık Anlatımı

* Şu an log() fonksiyonumuz sadece ekrana basıyor.
#### Ama biz:

* Aynı mesajı bir dosyaya da yazmak istiyoruz,

* Dosya otomatik oluşturulsun (logs/server.log),

* Her log’a tarih-saat ve seviye eklensin,

* Terminalde renkli kalsın, dosyada düz siyah-beyaz olsun.

* Yani terminal “renkli”, dosya “okunabilir sade” olacak.

## 🔍 Küçük Deneme (Örnek Mekanizma)

In [ ]:
import os
from datetime import datetime

LOG_DIR = "logs"
os.makedirs(LOG_DIR, exist_ok=True)
LOG_FILE = os.path.join(LOG_DIR, "server.log")

def log_to_file(msg):
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"[{now}] {msg}\n")

def log(msg, color="\033[94m", symbol="💬"):
    RESET = "\033[0m"
    now = datetime.now().strftime("%H:%M:%S")
    text = f"[{now}] {symbol} {msg}"
    print(f"{color}{text}{RESET}")
    log_to_file(text)

### 🎯 Avantajı

* Hatalar ve istek geçmişi kalıcı olur.

* Production’da hata yakalamak kolaylaşır.

* Log’ları analiz eden araçlara (Grafana, Kibana, vs.) beslenebilir.

## 🧩 AŞAMA 9 — Renkli + Dosyalı Loglama Entegreli API

In [ ]:
# ==========================================================
# 🧩 AŞAMA 9.7 — Renkli + Dosyalı Loglama Entegreli API
# ==========================================================

import socket
import threading
import json
import os
import mimetypes
from datetime import datetime
from typing import Any

# ---- Genel Ayarlar ----
HOST = "127.0.0.1"
PORT = 8097
BUF = 4096
STATIC_DIR = r"C:\Users\hdgn5\OneDrive\Masaüstü\Kendi API'mız\- API Geliştirne Adımları -\Part 3"

# Log klasörü ve dosyası
LOG_DIR = "logs"
os.makedirs(LOG_DIR, exist_ok=True)
LOG_FILE = os.path.join(LOG_DIR, "server.log")

mimetypes.init()

# ==========================================================
# 🎨 1️⃣ Renk Kodları + Log Fonksiyonları
# ==========================================================
RESET  = "\033[0m"
RED    = "\033[91m"
GREEN  = "\033[92m"
YELLOW = "\033[93m"
BLUE   = "\033[94m"
CYAN   = "\033[96m"

def log_to_file(msg: str):
    """Logları 'logs/server.log' dosyasına kaydeder."""
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"[{now}] {msg}\n")

def log(msg: str, color=BLUE, symbol="💬"):
    """Terminal + dosya logu."""
    now = datetime.now().strftime("%H:%M:%S")
    text = f"[{now}] {symbol} {msg}"
    print(f"{color}{text}{RESET}")
    log_to_file(text)

# ==========================================================
# 2️⃣ Dinamik Route Sistemi
# ==========================================================
ROUTES = {}

def route(method, path):
    def decorator(func):
        ROUTES[(method.upper(), path)] = func
        log(f"Route eklendi → {method.upper()} {path} -> {func.__name__}", CYAN, "📍")
        return func
    return decorator

# ==========================================================
# 3️⃣ Auto Response (dict→JSON, str→HTML, bytes→Binary)
# ==========================================================
def make_response_auto(data: Any):
    if isinstance(data, tuple):
        return data
    if isinstance(data, dict):
        return 200, "application/json", json.dumps(data, ensure_ascii=False)
    if isinstance(data, str):
        return 200, "text/html", data
    if isinstance(data, (bytes, bytearray)):
        return 200, "application/octet-stream", bytes(data)
    return 500, "application/json", json.dumps({"error": f"Unsupported type: {type(data)}"}, ensure_ascii=False)

def build_http_response(status: int, ctype: str, body):
    if isinstance(body, str):
        body_bytes = body.encode("utf-8")
    elif isinstance(body, (bytes, bytearray)):
        body_bytes = bytes(body)
    else:
        body_bytes = json.dumps(body, ensure_ascii=False).encode("utf-8")
        ctype = "application/json"

    status_text = {200:"OK",400:"Bad Request",404:"Not Found",500:"Internal Server Error"}.get(status, "OK")

    headers = (
        f"HTTP/1.1 {status} {status_text}\r\n"
        f"Content-Type: {ctype}; charset=utf-8\r\n"
        f"Content-Length: {len(body_bytes)}\r\n"
        f"Connection: close\r\n"
        f"\r\n"
    ).encode("ascii")

    return headers + body_bytes

# ==========================================================
# 4️⃣ Global Error Handler
# ==========================================================
def global_error_handler(func, *args):
    try:
        return func(*args)
    except FileNotFoundError as e:
        log(f"404 Not Found → {e}", YELLOW, "⚠️")
        return (404, "application/json", json.dumps({"error": str(e), "type": "FileNotFoundError"}, ensure_ascii=False))
    except ValueError as e:
        log(f"400 Bad Request → {e}", YELLOW, "⚠️")
        return (400, "application/json", json.dumps({"error": str(e), "type": "ValueError"}, ensure_ascii=False))
    except KeyError as e:
        log(f"Eksik anahtar → {e}", YELLOW, "⚠️")
        return (400, "application/json", json.dumps({"error": f"Eksik anahtar: {e}", "type": "KeyError"}, ensure_ascii=False))
    except Exception as e:
        log(f"500 Internal Error → {type(e).__name__}: {e}", RED, "❌")
        return (500, "application/json", json.dumps({"error": str(e), "type": type(e).__name__}, ensure_ascii=False))

# ==========================================================
# 5️⃣ HTTP Parsing
# ==========================================================
def parse_request(conn):
    data = b""
    while b"\r\n\r\n" not in data:
        chunk = conn.recv(BUF)
        if not chunk:
            break
        data += chunk

    headers_part, _, body_bytes = data.partition(b"\r\n\r\n")
    lines = headers_part.decode("utf-8", errors="ignore").split("\r\n")
    method, path, _ = lines[0].split()
    headers = {}
    for line in lines[1:]:
        if ":" in line:
            key, value = line.split(":", 1)
            headers[key.strip().lower()] = value.strip()

    content_length = int(headers.get("content-length", "0")) if headers else 0
    while len(body_bytes) < content_length:
        body_bytes += conn.recv(BUF)

    body = body_bytes.decode("utf-8", errors="ignore")
    return method, path.rstrip("/") or "/", headers, body

# ==========================================================
# 6️⃣ Endpoint’ler
# ==========================================================
@route("GET", "/")
def root():
    return "<h1>📜 Log Sistemi 9.7</h1><p>Renkli + Dosyalı log aktif 🎨</p>"

@route("POST", "/sum")
def sum_numbers(headers, body):
    data = json.loads(body)
    a, b = float(data["a"]), float(data["b"])
    return {"sum": a + b}

@route("POST", "/sum/plain")
def sum_plain(headers, body):
    data = json.loads(body)
    a, b = float(data["a"]), float(data["b"])
    return (200, "text/plain", str(a + b))

# ==========================================================
# 7️⃣ Statik Dosya Servisi
# ==========================================================
def serve_static(path):
    if path in ("", "/"):
        path = "text.html"
    full_path = os.path.join(STATIC_DIR, path.lstrip("/"))
    if not os.path.exists(full_path):
        raise FileNotFoundError(f"{path} bulunamadı.")
    ctype, _ = mimetypes.guess_type(full_path)
    if not ctype:
        ctype = "application/octet-stream"
    with open(full_path, "rb") as f:
        body = f.read()
    return 200, ctype, body

# ==========================================================
# 8️⃣ İstek İşleyici
# ==========================================================
def handle_client(conn, addr):
    log(f"Yeni bağlantı: {addr}", BLUE, "📡")
    try:
        method, path, headers, body = parse_request(conn)
        key = (method.upper(), path)
        log(f"İstek geldi → {key}", YELLOW, "🔎")

        if key in ROUTES:
            handler = ROUTES[key]
            raw = global_error_handler(handler, headers, body) if method == "POST" else global_error_handler(handler)
        else:
            raw = global_error_handler(serve_static, path)

        status, ctype, resp_body = make_response_auto(raw)
        log(f"{status} → {path}", GREEN, "✅")
    except Exception as e:
        log(f"Sunucu hatası: {e}", RED, "💥")
        status, ctype, resp_body = 500, "application/json", json.dumps({"error": str(e)}, ensure_ascii=False)

    conn.sendall(build_http_response(status, ctype, resp_body))
    conn.close()
    log(f"Bağlantı kapatıldı: {addr}", CYAN, "🔚")

# ==========================================================
# 9️⃣ Sunucu Başlatma
# ==========================================================
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as srv:
    srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    srv.bind((HOST, PORT))
    srv.listen(64)
    log(f"Renkli + Dosyalı Log API çalışıyor → http://{HOST}:{PORT}", CYAN, "🚀")

    while True:
        conn, addr = srv.accept()
        t = threading.Thread(target=handle_client, args=(conn, addr))
        t.start()


## Bir de süre ekleyelim ve daha profesyonelce olsun

In [ ]:
# ==========================================================
# 🧩 AŞAMA 9 — Renkli + Dosyalı Log + Süre Ölçümü (X-Response-Time)
# ==========================================================
# Özellikler:
# - Dinamik route dekoratörü
# - Auto Response (dict→JSON, str→HTML, bytes→Binary)
# - Global Error Handler
# - Statik dosya sunumu (binary, doğru MIME)
# - Doğru Content-Length (byte cinsinden)
# - Renkli terminal logları + logs/server.log dosyasına kalıcı kayıt
# - ⏱ İstek süresi ölçümü (ms) + "X-Response-Time" header’ı
# ==========================================================

import socket
import threading
import json
import os
import mimetypes
import time
from datetime import datetime
from typing import Any, Dict, Tuple, Optional

# ---- Genel Ayarlar ----
HOST = "127.0.0.1"
PORT = 8098
BUF = 4096
STATIC_DIR = r"C:\Users\hdgn5\OneDrive\Masaüstü\Kendi API'mız\- API Geliştirne Adımları -\Part 3"

# Log klasörü ve dosyası
LOG_DIR = "logs"
os.makedirs(LOG_DIR, exist_ok=True)
LOG_FILE = os.path.join(LOG_DIR, "server.log")

mimetypes.init()

# ==========================================================
# 🎨 Renk Kodları + Log Fonksiyonları
# ==========================================================
RESET  = "\033[0m"
RED    = "\033[91m"
GREEN  = "\033[92m"
YELLOW = "\033[93m"
BLUE   = "\033[94m"
CYAN   = "\033[96m"

def _file_log_line(msg: str) -> str:
    return f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}"

def log_to_file(msg: str):
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(_file_log_line(msg) + "\n")

def log(msg: str, color=BLUE, symbol="💬"):
    line = f"[{datetime.now().strftime('%H:%M:%S')}] {symbol} {msg}"
    print(f"{color}{line}{RESET}")
    log_to_file(line)

# ==========================================================
# Dinamik Route Sistemi
# ==========================================================
ROUTES: Dict[Tuple[str, str], callable] = {}

def route(method: str, path: str):
    def decorator(func):
        ROUTES[(method.upper(), path)] = func
        log(f"Route eklendi → {method.upper()} {path} -> {func.__name__}", CYAN, "📍")
        return func
    return decorator

# ==========================================================
# Auto Response (dict→JSON, str→HTML, bytes→Binary)
# ==========================================================
def make_response_auto(data: Any) -> Tuple[int, str, Any]:
    if isinstance(data, tuple):
        return data
    if isinstance(data, dict):
        return 200, "application/json", json.dumps(data, ensure_ascii=False)
    if isinstance(data, str):
        return 200, "text/html", data
    if isinstance(data, (bytes, bytearray)):
        return 200, "application/octet-stream", bytes(data)
    return 500, "application/json", json.dumps({"error": f"Unsupported type: {type(data)}"}, ensure_ascii=False)

def build_http_response(status: int, ctype: str, body: Any, extra_headers: Optional[Dict[str, str]]=None) -> bytes:
    # Body → bytes
    if isinstance(body, str):
        body_bytes = body.encode("utf-8")
    elif isinstance(body, (bytes, bytearray)):
        body_bytes = bytes(body)
    else:
        body_bytes = json.dumps(body, ensure_ascii=False).encode("utf-8")
        ctype = "application/json"

    status_text = {200:"OK",400:"Bad Request",404:"Not Found",500:"Internal Server Error"}.get(status, "OK")

    header_lines = [
        f"HTTP/1.1 {status} {status_text}",
        f"Content-Type: {ctype}; charset=utf-8",
        f"Content-Length: {len(body_bytes)}",
        "Connection: close",
    ]
    if extra_headers:
        for k, v in extra_headers.items():
            header_lines.append(f"{k}: {v}")

    headers = ("\r\n".join(header_lines) + "\r\n\r\n").encode("ascii")
    return headers + body_bytes

# ==========================================================
# Global Error Handler
# ==========================================================
def global_error_handler(func, *args):
    try:
        return func(*args)
    except FileNotFoundError as e:
        log(f"404 Not Found → {e}", YELLOW, "⚠️")
        return (404, "application/json", json.dumps({"error": str(e), "type": "FileNotFoundError"}, ensure_ascii=False))
    except ValueError as e:
        log(f"400 Bad Request → {e}", YELLOW, "⚠️")
        return (400, "application/json", json.dumps({"error": str(e), "type": "ValueError"}, ensure_ascii=False))
    except KeyError as e:
        log(f"Eksik anahtar → {e}", YELLOW, "⚠️")
        return (400, "application/json", json.dumps({"error": f"Eksik anahtar: {e}", "type": "KeyError"}, ensure_ascii=False))
    except Exception as e:
        log(f"500 Internal Error → {type(e).__name__}: {e}", RED, "❌")
        return (500, "application/json", json.dumps({"error": str(e), "type": type(e).__name__}, ensure_ascii=False))

# ==========================================================
# HTTP Parsing (Content-Length destekli)
# ==========================================================
def parse_request(conn: socket.socket):
    data = b""
    while b"\r\n\r\n" not in data:
        chunk = conn.recv(BUF)
        if not chunk:
            break
        data += chunk

    headers_part, _, body_bytes = data.partition(b"\r\n\r\n")
    lines = headers_part.decode("utf-8", errors="ignore").split("\r\n")
    if not lines or lines[0].strip() == "":
        return "GET", "/", {}, ""

    method, path, _ = lines[0].split()
    headers = {}
    for line in lines[1:]:
        if ":" in line:
            key, value = line.split(":", 1)
            headers[key.strip().lower()] = value.strip()

    content_length = int(headers.get("content-length", "0")) if headers else 0
    while len(body_bytes) < content_length:
        body_bytes += conn.recv(BUF)

    body = body_bytes.decode("utf-8", errors="ignore")
    path = (path or "/").rstrip("/") or "/"
    return method, path, headers, body

# ==========================================================
# Endpoint’ler
# ==========================================================
@route("GET", "/")
def root():
    return "<h1>⏱ Aşama 9.8</h1><p>Renkli log + dosyaya kayıt + süre ölçümü aktif. (X-Response-Time)</p>"

@route("POST", "/sum")
def sum_numbers(headers, body):
    data = json.loads(body)
    a, b = float(data["a"]), float(data["b"])
    return {"sum": a + b}

@route("POST", "/sum/plain")
def sum_plain(headers, body):
    data = json.loads(body)
    a, b = float(data["a"]), float(data["b"])
    return (200, "text/plain", str(a + b))

# ==========================================================
# Statik Dosya Servisi (binary okuma + doğru MIME)
# ==========================================================
def serve_static(path: str):
    if path in ("", "/"):
        path = "text.html"
    full_path = os.path.join(STATIC_DIR, path.lstrip("/"))
    if not os.path.exists(full_path) or not os.path.isfile(full_path):
        raise FileNotFoundError(f"{path} bulunamadı.")
    ctype, _ = mimetypes.guess_type(full_path)
    if not ctype:
        ctype = "application/octet-stream"
    with open(full_path, "rb") as f:
        body = f.read()
    return 200, ctype, body

# ==========================================================
# İstek İşleyici (Thread başına) + Süre Ölçümü
# ==========================================================
def handle_client(conn, addr):
    start = time.perf_counter()
    log(f"Yeni bağlantı: {addr}", BLUE, "📡")

    status, ctype, resp_body = 500, "application/json", json.dumps({"error": "uninitialized"}, ensure_ascii=False)
    extra_headers = {}

    try:
        method, path, headers, body = parse_request(conn)
        key = (method.upper(), path)
        log(f"İstek → {key}", YELLOW, "🔎")

        if key in ROUTES:
            handler = ROUTES[key]
            raw = global_error_handler(handler, headers, body) if method.upper() == "POST" else global_error_handler(handler)
        else:
            raw = global_error_handler(serve_static, path)

        status, ctype, resp_body = make_response_auto(raw)

    except Exception as e:
        log(f"Sunucu hatası: {e}", RED, "💥")
        status, ctype, resp_body = 500, "application/json", json.dumps({"error": str(e)}, ensure_ascii=False)
    finally:
        elapsed_ms = (time.perf_counter() - start) * 1000.0
        extra_headers["X-Response-Time"] = f"{elapsed_ms:.2f}ms"
        conn.sendall(build_http_response(status, ctype, resp_body, extra_headers))
        conn.close()
        color = GREEN if status < 400 else (YELLOW if status < 500 else RED)
        log(f"{status} → {path} ⏱ {elapsed_ms:.2f}ms", color, "✅" if status < 400 else ("⚠️" if status < 500 else "❌"))

# ==========================================================
# Sunucu Başlatma
# ==========================================================
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as srv:
    srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    srv.bind((HOST, PORT))
    srv.listen(64)
    log(f"Aşama 9.8 çalışıyor → http://{HOST}:{PORT}", CYAN, "🚀")

    while True:
        conn, addr = srv.accept()
        t = threading.Thread(target=handle_client, args=(conn, addr))
        t.start()


[23:53:55] 📍 Route eklendi → GET / -> root
[23:53:55] 📍 Route eklendi → POST /sum -> sum_numbers
[23:53:55] 📍 Route eklendi → POST /sum/plain -> sum_plain
[23:53:55] 🚀 Aşama 9.8 çalışıyor → http://127.0.0.1:8098
[23:53:57] 📡 Yeni bağlantı: ('127.0.0.1', 51748)
[23:53:57] 📡 Yeni bağlantı: ('127.0.0.1', 52741)
[23:53:57] 🔎 İstek → ('GET', '/')
[23:53:57] ✅ 200 → / ⏱ 4.21ms
[23:53:57] 🔎 İstek → ('GET', '/favicon.ico')
[23:53:57] ⚠️ 404 Not Found → /favicon.ico bulunamadı.
[23:53:57] ⚠️ 404 → /favicon.ico ⏱ 115.28ms
[23:54:11] 📡 Yeni bağlantı: ('127.0.0.1', 55654)
[23:54:11] 📡 Yeni bağlantı: ('127.0.0.1', 54866)
[23:54:11] 🔎 İstek → ('GET', '/')
[23:54:11] ✅ 200 → / ⏱ 4.99ms
[23:54:17] 🔎 İstek → ('GET', '/sum')
[23:54:17] 📡 Yeni bağlantı: ('127.0.0.1', 60471)
[23:54:17] ⚠️ 404 Not Found → /sum bulunamadı.
[23:54:17] ⚠️ 404 → /sum ⏱ 5217.99ms
[23:54:21] 🔎 İstek → ('GET', '/docs')
[23:54:21] ⚠️ 404 Not Found → /docs bulunamadı.
[23:54:21] 📡 Yeni bağlantı: ('127.0.0.1', 64695)
[23:54:21] ⚠️ 4